# Выбор локации для скважины

**Краткая суть**

Необходимо построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль

**Описание**

Предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализировать возможную прибыль и риски техникой Bootstrap

**План выполнения проекта**

1. Загрузить и подготовьть данные.
2. Обучить разные модели.
3. Сделать выводы.

In [2]:
import pandas as pd
import numpy as np


from scipy import stats as st
from numpy.random import RandomState
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import warnings
 
warnings.filterwarnings('ignore')

## Загрузка и подготовка данных

In [3]:
try:
    data_1 = pd.read_csv('/datasets/geo_data_0.csv')
    data_2 = pd.read_csv('/datasets/geo_data_1.csv')
    data_3 = pd.read_csv('/datasets/geo_data_2.csv')
except: 
    data_1 = pd.read_csv('file_for_projects/geo_data_0.csv')
    data_2 = pd.read_csv('file_for_projects/geo_data_1.csv')
    data_3 = pd.read_csv('file_for_projects/geo_data_2.csv')

### Выводем информацию по нашим данным.

In [4]:
def checking(data):
    data.info()
    display(data.isna().sum())
    display(data.describe())

In [2]:
#checking(data_1)

In [3]:
#checking(data_2)

In [4]:
#checking(data_3)

Нам повезло, у нас 3 хороших датасета. 
Столбец product - целевой признак, столбец id - удалим, т к он не несёт никакой информации для модели, столбцы f0 f1 f2 - значимые признаки. 
Во всех трёх датасетах, есть связь между f0 и f1, f2 и product.  Распределение равномерное, критических выбросов нет.

### Выделим в наших датасетах обучающие и целевые признаки,  удалим столбец id. Разделим на тренировочные и валидационные датасеты.

In [8]:
def split_data(data):
    features, target = data.drop(['id', 'product'], axis=1), data['product']
    features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=22)
    return features, target, features_train, features_valid, target_train, target_valid

In [9]:
features_1, target_1, features_train_1, features_valid_1, target_train_1, target_valid_1 = split_data(data_1)

In [10]:
features_2, target_2, features_train_2, features_valid_2, target_train_2, target_valid_2 = split_data(data_2)

In [11]:
features_3, target_3, features_train_3, features_valid_3, target_train_3, target_valid_3 = split_data(data_3)

### Произведем нормализацию числовых признаков

In [12]:
numeric = ['f0', 'f1', 'f2']

In [13]:
def normalized(features_train, features_valid):
    scaler = StandardScaler()
    scaler.fit(features_train[numeric])
    features_train[numeric] = scaler.transform(features_train[numeric])
    features_valid[numeric] = scaler.transform(features_valid[numeric])
    display(features_train.shape, 
            features_train.info(),
            features_valid.shape,
            features_valid.info()
           )

In [5]:
#normalized(features_train_1, features_valid_1)

In [6]:
#normalized(features_train_2, features_valid_2)

In [7]:
#normalized(features_train_3, features_valid_3)

## Обучение и проверка модели

### Будем использовать линейную регрессию 

In [17]:
lg_1 = LinearRegression()
lg_2 = LinearRegression()
lg_3 = LinearRegression()

In [18]:
def learning_model(data, model, features_train, target_train, features_valid, target_valid):
    model.fit(features_train, target_train)
    predict_valid = model.predict(features_valid)
    predict_valid_mean = predict_valid.mean()
    rmse = mean_squared_error(target_valid, predict_valid) ** 0.5

    
    print("Средний запас предсказанного объема сырья в  регионе:", np.round(predict_valid_mean, 2))

    print("RMSE на валидационной выборке:", rmse) 

    
    return predict_valid, np.round(predict_valid_mean, 2), rmse


In [19]:
predict_valid_1, predict_valid_mean_1, rmse_1 = learning_model(data_1, lg_1, features_train_1, target_train_1, 
                                                              features_valid_1, target_valid_1)

Средний запас предсказанного объема сырья в  регионе: 92.54
RMSE на валидационной выборке: 37.411414310487665


In [20]:
predict_valid_2, predict_valid_mean_2, rmse_2 = learning_model(data_2, lg_2, features_train_2, target_train_2, 
                                                              features_valid_2, target_valid_2)

Средний запас предсказанного объема сырья в  регионе: 68.72
RMSE на валидационной выборке: 0.8916430396076015


In [21]:
predict_valid_3, predict_valid_mean_3, rmse_3 = learning_model(data_3, lg_3, features_train_3, target_train_3, 
                                                              features_valid_3, target_valid_3)

Средний запас предсказанного объема сырья в  регионе: 94.88
RMSE на валидационной выборке: 40.161072438148516


- для региона 1 модель предсказала (в среднем) 92 тыс. баррелей сырья, но также модель склонна ошибаться на 37 тыс. баррелей.
- для региона 2 модель предсказала (в среднем) 68 тыс. баррелей сырья, но здесь уже ошибка многократно ниже. 
- для региона 3 модель предсказала (в среднем) 94 тыс. баррелей сырья, но тут модель тоже склонна к ошибке на 40 тыс. баррелей.
- если посмотреть на цифры, то кажется, что нужно разворачивать бизнес в регионе 2, т к метрики качества модели стремятся к еденице, что говорит нам о её достоверности.


## Подготовка к расчёту прибыли

### Определим константы для расчёта

In [22]:
FULL_BUDGET = 10000 # бюджет в млн рублей
NEED_POINTS = 200 # количество разрабатываемых точек
PRICE_PRODUCT = 0.45 # доходность одной единицы барреля в млн рублей

### Рассчитаем нужные значения

In [23]:
price_for_one = FULL_BUDGET / NEED_POINTS # бюджет на одну скважину (млн. руб)
V_product = np.round(price_for_one / PRICE_PRODUCT, 2) # объём сырья из одной скважины для безубыточной разработки (тыс. сырья)


price_product_data_1 = PRICE_PRODUCT * predict_valid_mean_1 # средний доход из одной скважины в регионе 1 (млн. руб)
price_product_data_2 = PRICE_PRODUCT * predict_valid_mean_2 # средний доход из одной скважины в регионе 2 (млн. руб)
price_product_data_3 = PRICE_PRODUCT * predict_valid_mean_3 # средний доход из одной скважины в регионе 3 (млн. руб)

In [24]:
print('Объём сырья из одной скважины для безубыточной разработки:', V_product, 'тыс. сырья'),
print('Средний предсказанный запас сырья для региона 1:', predict_valid_mean_1, 'тыс. сырья'), 
print('Средний предсказанный запас сырья для региона 2:', predict_valid_mean_2, 'тыс. сырья'),
print('Средний предсказанный запас сырья для региона 3:', predict_valid_mean_3, 'тыс. сырья')

Объём сырья из одной скважины для безубыточной разработки: 111.11 тыс. сырья
Средний предсказанный запас сырья для региона 1: 92.54 тыс. сырья
Средний предсказанный запас сырья для региона 2: 68.72 тыс. сырья
Средний предсказанный запас сырья для региона 3: 94.88 тыс. сырья


In [25]:
print('Бюджет на одну скважину:', price_for_one, 'млн. руб'), 
print('Средний доход с одной скважины в регионе 1:', price_product_data_1, 'млн. руб'), 
print('Средний доход с одной скважины в регионе 2:', price_product_data_2, 'млн. руб'),
print('Средний доход с одной скважины в регионе 3:', price_product_data_3, 'млн. руб')

Бюджет на одну скважину: 50.0 млн. руб
Средний доход с одной скважины в регионе 1: 41.643 млн. руб
Средний доход с одной скважины в регионе 2: 30.924 млн. руб
Средний доход с одной скважины в регионе 3: 42.696 млн. руб


Судя по первоначальным рассчётам, стоит пересмотреть регионы для добычи, потому как ,кажется, кругом убытки для всех трех регионов. Выявим лучшие скважины, посмотрим как изменятся данные.

## Расчёт прибыли и рисков 

In [26]:
def profit_count(predictions, targets):
    best_predictions = pd.Series(predictions).sort_values(ascending=False)
    best_point = targets.iloc[best_predictions.index][:200]
    profit = (PRICE_PRODUCT * best_point.sum()) - FULL_BUDGET
    return profit, best_point, best_predictions

In [27]:
profit_1, best_point_1, best_predictions_1 = profit_count(predict_valid_1, target_valid_1)

In [28]:
print('Средний запас сырья для 200 лучших скважин для региона 1:', np.round(best_point_1.mean(), 2), 'тыс. сырья')
print('Прибыль для 200 лушчих скважин региона 1:', np.round(profit_1, 2), 'млн. руб')

Средний запас сырья для 200 лучших скважин для региона 1: 148.35 тыс. сырья
Прибыль для 200 лушчих скважин региона 1: 3351.54 млн. руб


In [29]:
profit_2, best_point_2, best_predictions_2 = profit_count(predict_valid_2, target_valid_2)

In [30]:
print('Средний запас сырья для 200 лучших скважин для региона 2:', np.round(best_point_2.mean(), 2), 'тыс. сырья')
print('Прибыль для 200 лушчих скважин региона 2:', np.round(profit_2, 2), 'млн. руб')

Средний запас сырья для 200 лучших скважин для региона 2: 137.95 тыс. сырья
Прибыль для 200 лушчих скважин региона 2: 2415.09 млн. руб


In [31]:
profit_3, best_point_3, best_predictions_3 = profit_count(predict_valid_3, target_valid_3)

In [32]:
print('Средний запас сырья для 200 лучших скважин для региона 3:', np.round(best_point_3.mean(), 2), 'тыс. сырья')
print('Прибыль для 200 лушчих скважин региона 3:', np.round(profit_3, 2), 'млн. руб')

Средний запас сырья для 200 лучших скважин для региона 3: 142.8 тыс. сырья
Прибыль для 200 лушчих скважин региона 3: 2852.28 млн. руб




1) Если взять 200 скважин с самым большим запасом сырья для каждого региона, и для них посчитать средний запас сырья, то мы увидим, что наши полученные значения выше минимального значения для безубыточной работы (111.11 тыс. сырья).

2) Самая большая прогнозируемая прибыль получается для региона 1 с 3351.54 млн. руб

3) На втором месте идёт регион 3 с 2852.28 млн. руб 

3) На третьем месте регион 2 с 2415.09 млн. руб.


In [33]:
state = np.random.RandomState(12345)

In [34]:
def quantile_count(predictions, targets):
    values = []
    for i in range(1000):
        subsample = pd.Series(predictions).sample(n=500, replace=True, random_state=state)
        values.append(profit_count(subsample, targets.reset_index(drop=True))[0])

    values = pd.Series(values)
    lower = values.quantile(0.025) 
    upper = values.quantile(0.975) 
    return lower, upper, values 

In [35]:
lower_1, upper_1, values_1 = quantile_count(predict_valid_1, target_valid_1)
lower_1, upper_1

(-53.9691699514447, 973.9879969634377)

In [36]:
lower_2, upper_2, values_2 = quantile_count(predict_valid_2, target_valid_2)
lower_2, upper_2

(31.00285695730218, 819.4930368420247)

In [37]:
lower_3, upper_3, values_3 = quantile_count(predict_valid_3, target_valid_3)
lower_3, upper_3

(-148.33970531475296, 916.4385427142356)

- доля прибыли из 1 скважины для региона 1 может варрироваться между убытком в 2 млн. руб и доходом в 973 млн. руб
- доля прибыли из 1 скважины для региона 2 может варрироваться между 37 млн. руб и в 803 млн. руб
- доля прибыли из 1 скважины для региона 3 может варрироваться между убытком в 106 млн. руб и доходом в 885 млн. руб

### Расчитаем вероятность убытков для каждого из регионов

In [38]:
def loss_probability(value):
    loss = ((value[value < 0]).count() / value.count()) * 100
    return loss

In [39]:
loss_1 = loss_probability(values_1)
loss_1

3.5999999999999996

In [40]:
loss_2 = loss_probability(values_2)
loss_2

1.9

In [41]:
loss_3 = loss_probability(values_3)
loss_3

8.0

In [42]:
print('Вероятность убытков для региона 1:', np.round(loss_1, 2),'%')
print('Вероятность убытков для региона 2:', np.round(loss_2, 2),'%')
print('Вероятность убытков для региона 3:', np.round(loss_3, 2),'%')

Вероятность убытков для региона 1: 3.6 %
Вероятность убытков для региона 2: 1.9 %
Вероятность убытков для региона 3: 8.0 %


# Вывод:
Нам нужно было определить регион, в котором развернуть добычу нефти было максимально выгодно и эффективно. Были предоставлены пробы нефти в трёх регионах, характеристики для каждой скважины в регионе были известны. Нами были построены модели для предсказания объёма сырья в скважинах для каждого региона. На первый взгляд, по среднему объёму нефти, казалось, что все три региона убыточны. 

Мы взяли 200 скважин в каждом регионе с самым максимальным запасом сырья, посчитали доход и получили, что самый самая большая прогнозируемая прибыль получается для региона 1 с 3351.54 млн. руб, на втором месте идёт регион 3 с 2852.28 млн. руб, на третьем месте регион 2 с 2415.09 млн. руб.

В регионах 1 и 3 могут встречаться убыточные скважины, а в регионе 2 все доходные. 

Условием к задаче было то, что вероятность убытков должна быть меньше 2.5 процентов, а это значит что только регион 2 попадает в эти рамки. В дополнение, метрики качества модели для региона 2 стремятся к еденице, что говорит нам о её достоверности.

